In [1]:
from data import FShapeFileReader, FShapeDirectoryReader

motif = FShapeFileReader("data/HNRNPC/hnrnpc_expected_pattern.txt").read()
sequences = FShapeDirectoryReader("data/HNRNPC/hnrnpc_binding_sites_fshape").read()

In [2]:
from tools import PromisingMotifsFinder

motif_length = len(motif)
promising_motifs = {length: [] for length in range(motif_length, motif_length + 3)}
for length in promising_motifs.keys():
    pmf = PromisingMotifsFinder(motif)
    for s in sequences:
        promising_motifs[length].extend(pmf.find(s, length))

In [ ]:
for key, value in promising_motifs.items():
    print(key, len(value))

In [ ]:
from tools import DBScanClusterer

db_clusters = {length: [] for length in promising_motifs.keys()}
epss = (0.8, 1.15, 1.5)
for length, eps in zip(promising_motifs.keys(), epss):
    db_clusters[length] = DBScanClusterer(eps=eps).transform(promising_motifs[length])

db_clusters

In [ ]:
from tools import AgglomerativeClusterer

db_clusters = {length: [] for length in promising_motifs.keys()}
for length in promising_motifs.keys():
    db_clusters[length] = AgglomerativeClusterer().transform(promising_motifs[length])

db_clusters

In [ ]:
from tools import ConsensusMotifs

consensus_motifs = ConsensusMotifs().compute_consensus(promising_motifs, db_clusters)
consensus_motifs